https://adventofcode.com/2024/day/6

In [76]:
from helpers import *
import re
import copy

In [113]:
lines = read_lines("input-06.txt")

## Part 1

First find the guard. This gives us the row and column of the guards position and the character representing them so we know the starting direction.

In [114]:
for row in range(0, len(lines)):
    m = re.search("[<>v^]", lines[row])
    if m:
        col = m.span()[0]
        guard = m.group()
        break
pos = (row, col)

if guard == "^":
    dir = (-1, 0)
elif guard == "v":
    dir = (1, 0)
elif guard == ">":
    dir = (0, 1)
elif guard == "<":
    dir = (-1, 0)
else:
    print("Don't recognise this guard!")

lab = []
for l in lines:
    lab.append([c for c in l])    
lab[row][col] = 'X'

Now to wander around the lab following the rules until we escape, counting steps as we go.

In [115]:
def escaped(p):
    return p[0] < 0 or p[1] < 0 or p[0] >= len(lines) or p[1] >= len(lines)

def patrol_route(start, dir, lab, limit):
    count = 1
    route = []
    pos = start
    next_pos = (pos[0] + dir[0], pos[1] + dir[1])
    
    while not escaped(next_pos):
        next_square = lab[next_pos[0]][next_pos[1]]
        #print(pos)
        if next_square != "#":
            pos = next_pos
            next_pos = (pos[0] + dir[0], pos[1] + dir[1])
            count += 1
            if count > limit:
                return []
            if next_square == '.':
                lab[pos[0]][pos[1]] = 'X'
                route.append((pos[0], pos[1]))
        else:
            dir = (dir[1], dir[0]*-1) # Rotate right 90
            next_pos = (pos[0] + dir[0], pos[1] + dir[1])
    
    return route


In [116]:
route = patrol_route(pos, dir, copy.deepcopy(lab), 10000)

print(len(route)+1)

5305


## Part two

Now to look for places for obstacles that would create a loop.

Not sure how to go about this, other than brute force. I guess there's only a point in trying places on the unobstructed route previously found - the guard is not going to visit any other places. So, I'll modify the above code to keep a list of those places and then here we can just try them all.

The next problem is how to detect a loop. For now, just counting steps and assuming a loop if we hit a limit. Most loops will be much shorter than that, I assume, and so something more intelligent might well be faster. Running on my Pi 4 with this simple loop detection, the whole process takes about 3 minutes. Long enough to be annoying but maybe not long enough to bother trying to optimise it! Perhaps if I find some enthusiasm somewhere...

Anyway, first reset everything.

In [117]:
for row in range(0, len(lines)):
    m = re.search("[<>v^]", lines[row])
    if m:
        col = m.span()[0]
        guard = m.group()
        break
pos = (row, col)

if guard == "^":
    dir = (-1, 0)
elif guard == "v":
    dir = (1, 0)
elif guard == ">":
    dir = (0, 1)
elif guard == "<":
    dir = (-1, 0)
else:
    print("Don't recognise this guard!")

lab = []
for l in lines:
    lab.append([c for c in l])    
lab[row][col] = 'X'

In [126]:
%%time

loop_count = 0

for p in route:
    #print("Checking:", p)
    lab2 = copy.deepcopy(lab)
    lab2[p[0]][p[1]] = "#"
    new_route = patrol_route(pos, dir, lab2, 10000)
    if len(new_route) == 0:
        loop_count += 1

print(loop_count)

2143
CPU times: user 3min 6s, sys: 483 ms, total: 3min 7s
Wall time: 3min 8s
